# Projeto Final — Análise Silver
## Análise de Reclamações Financeiras

### Objetivo da Camada Silver
Nesta camada realizamos análises exploratórias e descritivas para responder às perguntas de negócio:
- Perfil dos consumidores que mais reclamam
- Tipos de problemas mais recorrentes
- Desempenho das instituições financeiras
- Panorama geral de satisfação

Os insights gerados aqui servirão de base para a camada Gold (análises aprofundadas e comparativos estratégicos).

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

agibank_palette = ["#0064F5", "#77DF40", "#FFD600", "#0033B0", "#000F4A", "#C9FF90"]
sns.set_palette(agibank_palette)
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_context("talk")

plt.rcParams["axes.labelcolor"] = "#000F4A"
plt.rcParams["xtick.color"] = "#000F4A"
plt.rcParams["ytick.color"] = "#000F4A"
plt.rcParams["text.color"] = "#000F4A"

In [7]:
arquivo = "../../../data/silver/consumidor_gov_bronze_v2.csv"

df = pd.read_csv(arquivo, sep=";", encoding="utf-8", low_memory=False)

print("Dimensão da base Silver:", df.shape)
print(f"Carregados {df.shape[0]:,} registros e {df.shape[1]} colunas".replace(",", "."))

Dimensão da base Silver: (2567095, 30)
Carregados 2.567.095 registros e 30 colunas


In [8]:
import os
print(os.getcwd())

c:\Users\caroline.coutinho\projeto_mediacao_bancaria\analises\silver\notebooks_silver


In [9]:
print("Primeiras 5 linhas da base:\n")
display(df.head())

Primeiras 5 linhas da base:



,Região,UF,Cidade,Sexo,Faixa Etária,Ano Abertura,Mês Abertura,Data Abertura,Data Resposta,Data Finalização,...,Respondida,Situação,Avaliação Reclamação,Nota do Consumidor,data_source,file_origin,processed_at,file_month,is_agibank,quality_score
0,SE,MG,Belo Horizonte,M,entre 61 a 70 anos,2024,11,01/11/2024,11/11/2024,2025-01-20,...,S,Finalizada avaliada,Não Resolvida,1.0,consumidor_gov,basecompleta2025-01.csv,2026-02-04 13:03:01.384007,01/2025,False,0.9
1,CO,MT,Cuiabá,M,entre 31 a 40 anos,2024,11,01/11/2024,11/11/2024,2025-01-15,...,S,Finalizada avaliada,Não Resolvida,1.0,consumidor_gov,basecompleta2025-01.csv,2026-02-04 13:03:01.384007,01/2025,False,0.9
2,NE,SE,Aracaju,M,entre 21 a 30 anos,2024,11,01/11/2024,11/11/2024,2025-01-12,...,S,Finalizada avaliada,Não Resolvida,5.0,consumidor_gov,basecompleta2025-01.csv,2026-02-04 13:03:01.384007,01/2025,False,0.9
3,CO,MS,Campo Grande,F,entre 61 a 70 anos,2024,11,01/11/2024,08/11/2024,2025-01-27,...,S,Finalizada avaliada,Resolvida,5.0,consumidor_gov,basecompleta2025-01.csv,2026-02-04 13:03:01.384007,01/2025,False,0.9
4,SE,RJ,Rio de Janeiro,M,entre 41 a 50 anos,2024,11,01/11/2024,11/11/2024,2025-01-17,...,S,Finalizada avaliada,Não Resolvida,3.0,consumidor_gov,basecompleta2025-01.csv,2026-02-04 13:03:01.384007,01/2025,False,0.9


In [10]:
print("Informações gerais das colunas:\n")
df.info()

Informações gerais das colunas:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2567095 entries, 0 to 2567094
Data columns (total 30 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   Região                  object 
 1   UF                      object 
 2   Cidade                  object 
 3   Sexo                    object 
 4   Faixa Etária            object 
 5   Ano Abertura            int64  
 6   Mês Abertura            int64  
 7   Data Abertura           object 
 8   Data Resposta           object 
 9   Data Finalização        object 
 10  Prazo Resposta          object 
 11  Tempo Resposta          float64
 12  Nome Fantasia           object 
 13  Segmento de Mercado     object 
 14  Área                    object 
 15  Assunto                 object 
 16  Grupo Problema          object 
 17  Problema                object 
 18  Como Comprou Contratou  object 
 19  Procurou Empresa        object 
 20  Respondida              object 
 21

In [11]:
print("Estatísticas descritivas:\n")
display(df.describe())

Estatísticas descritivas:



,Ano Abertura,Mês Abertura,Tempo Resposta,Nota do Consumidor,quality_score
count,2.567095e+06,2.567095e+06,2.235303e+06,765889.000000,2.567095e+06
mean,2.024957e+03,6.896649e+00,6.217775e+00,2.590886,9.000000e-01
std,2.018845e-01,3.321249e+00,3.285802e+00,1.763245,3.330670e-16
min,2.024000e+03,1.000000e+00,0.000000e+00,1.000000,9.000000e-01
25%,2.025000e+03,4.000000e+00,3.000000e+00,1.000000,9.000000e-01
50%,2.025000e+03,7.000000e+00,7.000000e+00,1.000000,9.000000e-01
75%,2.025000e+03,1.000000e+01,9.000000e+00,5.000000,9.000000e-01
max,2.025000e+03,1.200000e+01,3.100000e+01,5.000000,9.000000e-01


In [12]:
nulos = df.isna().mean().sort_values(ascending=False) * 100

print("Top 10 colunas com mais valores nulos:\n")
display(nulos.head(10))

Top 10 colunas com mais valores nulos:



Nota do Consumidor      70.165148
Tempo Resposta          12.924804
Data Resposta           12.924804
Avaliação Reclamação    10.193273
Sexo                     0.002142
Cidade                   0.000000
Ano Abertura             0.000000
Faixa Etária             0.000000
Data Abertura            0.000000
Mês Abertura             0.000000
dtype: float64

In [13]:
print("=" * 80)
print("ANALISE DE COLUNAS TEMPORAIS")
print("=" * 80)

palavras_chave = ['ano', 'mês', 'mes', 'data', 'prazo', 'tempo', 'processed_at', 'file_month']

colunas_temporais = []
for col in df.columns:
    col_lower = col.lower()
    if any(palavra in col_lower for palavra in palavras_chave):
        if 'source' not in col_lower:
            colunas_temporais.append(col)

print(f"\nColunas temporais encontradas: {len(colunas_temporais)}")
print("-" * 80)
for col in colunas_temporais:
    print(f"   {col} ({df[col].dtype})")

problemas = []

for col in colunas_temporais:
    tipo_atual = str(df[col].dtype)
    
    if col in ['Data Abertura', 'Data Resposta', 'Data Finalização', 'Prazo Resposta', 'processed_at']:
        if tipo_atual != 'datetime64[ns]':
            problemas.append({
                'Coluna': col, 
                'Tipo Atual': tipo_atual, 
                'Tipo Esperado': 'datetime64[ns]'
            })

print("\n\nPROBLEMAS IDENTIFICADOS")
print("-" * 80)

if problemas:
    print(f"\n{len(problemas)} colunas precisam ser corrigidas:\n")
    
    df_problemas = pd.DataFrame(problemas)
    df_problemas.index = range(1, len(df_problemas) + 1)
    
    display(df_problemas)
    
    print("\nColunas que estao CORRETAS e NAO precisam ser alteradas:")
    print("   Ano Abertura (int64)")
    print("   Mes Abertura (int64)")
    print("   Tempo Resposta (float64)")
    print("   file_month (object)")
    
else:
    print("\nTodas as colunas temporais estao corretas!")

print("\n" + "=" * 80)

ANALISE DE COLUNAS TEMPORAIS

Colunas temporais encontradas: 9
--------------------------------------------------------------------------------
   Ano Abertura (int64)
   Mês Abertura (int64)
   Data Abertura (object)
   Data Resposta (object)
   Data Finalização (object)
   Prazo Resposta (object)
   Tempo Resposta (float64)
   processed_at (object)
   file_month (object)


PROBLEMAS IDENTIFICADOS
--------------------------------------------------------------------------------

5 colunas precisam ser corrigidas:



,Coluna,Tipo Atual,Tipo Esperado
1,Data Abertura,object,datetime64[ns]
2,Data Resposta,object,datetime64[ns]
3,Data Finalização,object,datetime64[ns]
4,Prazo Resposta,object,datetime64[ns]
5,processed_at,object,datetime64[ns]



Colunas que estao CORRETAS e NAO precisam ser alteradas:
   Ano Abertura (int64)
   Mes Abertura (int64)
   Tempo Resposta (float64)
   file_month (object)



In [14]:
print("=" * 80)
print("ANALISE DE TIPOS DE DADOS - TODAS AS COLUNAS")
print("=" * 80)

print(f"\nTotal de colunas: {len(df.columns)}")
print(f"Total de registros: {len(df):,}")

analise_colunas = []

for col in df.columns:
    tipo = str(df[col].dtype)
    nao_nulos = df[col].notna().sum()
    nulos = df[col].isna().sum()
    perc_nulos = round((nulos / len(df)) * 100, 2)
    valores_unicos = df[col].nunique()
    amostra = str(df[col].dropna().iloc[0]) if df[col].notna().any() else 'N/A'
    
    analise_colunas.append({
        'Coluna': col,
        'Tipo': tipo,
        'Nulos': nulos,
        '% Nulos': perc_nulos,
        'Valores Unicos': valores_unicos,
        'Exemplo': amostra[:50]
    })

df_analise = pd.DataFrame(analise_colunas)

print("\n")
display(df_analise)

print("\n" + "=" * 80)
print("RESUMO POR TIPO DE DADO")
print("=" * 80)

tipos_resumo = df.dtypes.value_counts()
print("\n")
for tipo, qtd in tipos_resumo.items():
    print(f"{tipo}: {qtd} colunas")

print("\n" + "=" * 80)

ANALISE DE TIPOS DE DADOS - TODAS AS COLUNAS

Total de colunas: 30
Total de registros: 2,567,095




,Coluna,Tipo,Nulos,% Nulos,Valores Unicos,Exemplo
0,Região,object,0,0.00,5,SE
1,UF,object,0,0.00,27,MG
2,Cidade,object,0,0.00,6633,Belo Horizonte
3,Sexo,object,55,0.00,3,M
4,Faixa Etária,object,0,0.00,7,entre 61 a 70 anos
5,Ano Abertura,int64,0,0.00,2,2024
6,Mês Abertura,int64,0,0.00,12,11
7,Data Abertura,object,0,0.00,425,01/11/2024
8,Data Resposta,object,331792,12.92,408,11/11/2024
9,Data Finalização,object,0,0.00,365,2025-01-20



RESUMO POR TIPO DE DADO


object: 24 colunas
float64: 3 colunas
int64: 2 colunas
bool: 1 colunas



In [15]:
print("=" * 80)
print("RELATORIO DE QUALIDADE DE DADOS - TIPOS E VALORES")
print("=" * 80)

print("\n" + "=" * 80)
print("PROBLEMAS CRITICOS - DEVEM SER CORRIGIDOS")
print("=" * 80)

criticos = []

for col in df.columns:
    tipo_atual = str(df[col].dtype)
    valores_unicos = df[col].nunique()
    
    if tipo_atual == 'object':
        valores_com_espaco = df[col].dropna().astype(str).str.strip() != df[col].dropna().astype(str)
        if valores_com_espaco.any():
            exemplos = df[col][df[col].notna()].unique()[:5]
            criticos.append({
                'Coluna': col,
                'Tipo Atual': tipo_atual,
                'Problema': f'Valores com espaços extras',
                'Acao Necessaria': 'Aplicar .str.strip() para remover espaços'
            })
    
    if tipo_atual == 'object' and valores_unicos > 10:
        amostra = str(df[col].dropna().iloc[0])
        if '/' in amostra or '-' in amostra:
            tem_numeros = any(char.isdigit() for char in amostra)
            if tem_numeros and len(amostra) >= 8:
                criticos.append({
                    'Coluna': col,
                    'Tipo Atual': tipo_atual,
                    'Problema': f'Parece ser data mas esta como texto. Exemplo: {amostra}',
                    'Acao Necessaria': 'Converter para datetime64[ns]'
                })
    
    if tipo_atual == 'float64':
        valores_nao_nulos = df[col].dropna()
        if len(valores_nao_nulos) > 0:
            sao_inteiros = (valores_nao_nulos % 1 == 0).all()
            if sao_inteiros and valores_unicos <= 20:
                valores = sorted(df[col].dropna().unique())
                criticos.append({
                    'Coluna': col,
                    'Tipo Atual': tipo_atual,
                    'Problema': f'Representa valores inteiros {valores} mas esta como float',
                    'Acao Necessaria': 'Converter para int64 (tratar nulos)'
                })
    
    if valores_unicos == 1:
        valor = df[col].unique()[0]
        criticos.append({
            'Coluna': col,
            'Tipo Atual': tipo_atual,
            'Problema': f'Todos registros tem valor fixo: {valor}',
            'Acao Necessaria': 'Verificar se coluna esta sendo calculada/preenchida corretamente'
        })

if criticos:
    df_criticos = pd.DataFrame(criticos)
    df_criticos.index = range(1, len(df_criticos) + 1)
    print()
    display(df_criticos)
else:
    print("\nNenhum problema critico encontrado!")

print("\n" + "=" * 80)
print("MELHORIAS OPCIONAIS - OTIMIZACAO")
print("=" * 80)

opcionais = []

for col in df.columns:
    tipo_atual = str(df[col].dtype)
    valores_unicos = df[col].nunique()
    
    if tipo_atual == 'object' and valores_unicos == 2:
        valores = set(df[col].dropna().unique())
        if valores == {'S', 'N'}:
            opcionais.append({
                'Coluna': col,
                'Tipo Atual': tipo_atual,
                'Sugestao': 'Converter S/N para boolean (True/False)',
                'Beneficio': 'Melhor performance e clareza'
            })
    
    if tipo_atual == 'object' and 2 <= valores_unicos <= 50:
        tamanho_atual = df[col].memory_usage(deep=True) / 1024 / 1024
        if tamanho_atual > 1:
            opcionais.append({
                'Coluna': col,
                'Tipo Atual': tipo_atual,
                'Sugestao': f'Converter para category ({valores_unicos} valores unicos)',
                'Beneficio': f'Economia de memoria (~{tamanho_atual:.1f}MB)'
            })

if opcionais:
    df_opcionais = pd.DataFrame(opcionais)
    df_opcionais.index = range(1, len(df_opcionais) + 1)
    print()
    display(df_opcionais)
else:
    print("\nNenhuma melhoria opcional identificada")

print("\n" + "=" * 80)
print("RESUMO EXECUTIVO")
print("=" * 80)
print(f"\nTotal de problemas criticos: {len(criticos)}")
print(f"Total de melhorias opcionais: {len(opcionais)}")

print("\n" + "=" * 80)

RELATORIO DE QUALIDADE DE DADOS - TIPOS E VALORES

PROBLEMAS CRITICOS - DEVEM SER CORRIGIDOS



,Coluna,Tipo Atual,Problema,Acao Necessaria
1,Região,object,Valores com espaços extras,Aplicar .str.strip() para remover espaços
2,Data Abertura,object,Parece ser data mas esta como texto. Exemplo: ...,Converter para datetime64[ns]
3,Data Resposta,object,Parece ser data mas esta como texto. Exemplo: ...,Converter para datetime64[ns]
4,Data Finalização,object,Parece ser data mas esta como texto. Exemplo: ...,Converter para datetime64[ns]
5,Prazo Resposta,object,Parece ser data mas esta como texto. Exemplo: ...,Converter para datetime64[ns]
6,Nome Fantasia,object,Valores com espaços extras,Aplicar .str.strip() para remover espaços
7,Área,object,Valores com espaços extras,Aplicar .str.strip() para remover espaços
8,Problema,object,Valores com espaços extras,Aplicar .str.strip() para remover espaços
9,Nota do Consumidor,float64,"Representa valores inteiros [np.float64(1.0), ...",Converter para int64 (tratar nulos)
10,data_source,object,Todos registros tem valor fixo: consumidor_gov,Verificar se coluna esta sendo calculada/preen...



MELHORIAS OPCIONAIS - OTIMIZACAO



,Coluna,Tipo Atual,Sugestao,Beneficio
1,Região,object,Converter para category (5 valores unicos),Economia de memoria (~124.9MB)
2,UF,object,Converter para category (27 valores unicos),Economia de memoria (~124.9MB)
3,Sexo,object,Converter para category (3 valores unicos),Economia de memoria (~122.4MB)
4,Faixa Etária,object,Converter para category (7 valores unicos),Economia de memoria (~164.9MB)
5,Segmento de Mercado,object,Converter para category (45 valores unicos),Economia de memoria (~361.0MB)
6,Área,object,Converter para category (14 valores unicos),Economia de memoria (~250.5MB)
7,Grupo Problema,object,Converter para category (8 valores unicos),Economia de memoria (~234.3MB)
8,Como Comprou Contratou,object,Converter para category (9 valores unicos),Economia de memoria (~183.2MB)
9,Procurou Empresa,object,Converter S/N para boolean (True/False),Melhor performance e clareza
10,Procurou Empresa,object,Converter para category (2 valores unicos),Economia de memoria (~122.4MB)



RESUMO EXECUTIVO

Total de problemas criticos: 13
Total de melhorias opcionais: 17

